In [ ]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import os
import numpy as np
import time
import matplotlib.pyplot as plt

!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"
!wget "https://pjreddie.com/media/files/yolov3.weights"
!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"

# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join(["/content","coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")
 
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join(["/content", "yolov3.weights"])
configPath = os.path.sep.join(["/content", "yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
detector = cv2.dnn.readNetFromDarknet('/content/yolov3.cfg', '/content/yolov3.weights')
#detector = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [ ]:
# For Video
from google.colab.patches import cv2_imshow

video_capture = cv2.VideoCapture("/content/drive/MyDrive/demo (online-video-cutter.com).mp4")

# determine only the *output* layer names that we need from YOLO
ln = detector.getLayerNames()
ln = [ln[i - 1] for i in detector.getUnconnectedOutLayers()]

# video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    re,img = video_capture.read()
    try:
       img = cv2.resize(img, (1400, 1000), interpolation=cv2.INTER_AREA)
       print(img.shape)
    except:
        break
    height, width, channels = img.shape


    # USing blob function of opencv to preprocess image
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    #Detecting objects
    detector.setInput(blob)
    outs = detector.forward(ln)


    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    #We use NMS function in opencv to perform Non-maximum Suppression
    #we give it score threshold and nms threshold as arguments.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(LABELS), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(LABELS[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 2, color, 3)

    plt.imshow(img)
    # plt.imshow("Image",cv2.resize(img, (800,600)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()


In [ ]:
# # Capture frame-by-frame
image = cv2.imread("/content/drive/MyDrive/ObjectTracking/images/soccer.jpg")
height, width, channels = image.shape
plt.imshow(image)

# image = cv2.imread("/content/drive/MyDrive/ObjectTracking/images/living_room.jpg")
# height, width, channels = image.shape
# plt.imshow(image)

# image = cv2.imread("/content/drive/MyDrive/ObjectTracking/images/9th-food.jpg")
# height, width, channels = image.shape
# plt.imshow(image)


In [ ]:
# conversion of BGR to grayscale is necessary o atpply this operation
img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# determine only the *output* layer names that we need from YOLO
ln = detector.getLayerNames()
ln = [ln[i - 1] for i in detector.getUnconnectedOutLayers()]


# USing blob function of opencv to preprocess image
blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),swapRB=True, crop=False)
#Detecting objects
detector.setInput(blob)
outs = detector.forward(ln)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
  #  ### We use NMS function in opencv to perform Non-maximum Suppression
  #  ### we give it score threshold and nms threshold as arguments.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    colors = np.random.uniform(0, 255, size=(len(LABELS), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(LABELS[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y -5),cv2.FONT_HERSHEY_SIMPLEX,
			1/2, color, 2)


plt.imshow(img)
plt.axis("off")

In [ ]:
# Counting the number of personss
box, label, c_score= cv.detect_common_objects(image)
output= draw_bbox(image, box, label, c_score)
plt.show()
print('Number of persons in this image : ', label.count('person'))